# Eliminación de preguntas que no cumplen los criterios
Con el dataset limpio, podemos comenzar a realizar un análisis utilizando algunas técnicas de NLP para eliminar aquellas preguntas que no cumplan con los criterios del INE establecidos en el punto

El resultado de este paso serán:
- Un archivo que contenga el texto de las preguntas que no cumplen los criterios y una breve explicación de porqué no está cumpliéndolos (discurso de odio, violencia, malas palabras, etc.)

- Un archivo con las preguntas restantes (los campos serán los mismos que en el punto 3.1)

- ID del registro

- Entidad de Origen

- Edad

- Género

- Identificación con algún grupo en situación de discriminación

- Tema de la pregunta

- Texto de la pregunta

In [2]:
#!pip install pysentimiento

In [3]:
import pandas as pd

from pysentimiento import create_analyzer

hate_speech_analyzer = create_analyzer(task="hate_speech", lang="es")

limpio_df = pd.read_csv('./Base_limpia.csv')

for pregunta in limpio_df['pregunta']:
    analizador = hate_speech_analyzer.predict(pregunta).output
    if len(analizador) > 0:
      with open('./preguntas_rechazadas.txt', 'a') as archivo:
        archivo.write(f'La pregunta "{pregunta}" fue rechazada por: {analizador}')
      limpio_df = limpio_df[limpio_df['pregunta'] != pregunta]

limpio_df.to_csv('./Base_limpia.csv', index=False)

Tal vez está pendiente limpiar preguntas con malas palabras utilizando alguna base de dato de malas palabras en español, no lo he probado